In [ ]:
#szükséges könyvtárak importálása

#from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import torchvision
from torchvision import datasets, models, transforms
from torchvision.datasets import ImageFolder

import torch.backends.cudnn as cudnn
import numpy as np


import matplotlib.pyplot as plt
import time
import math
import os
import copy

#cuda-ra tesszük a hálót, ha az elérhető
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
#google drive csatlakoztatása

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from torchvision.transforms.transforms import RandomRotation
class_num = 6   #ennyi osztályunk van

batch_size = class_num ** 2

#adat augmentálás/sokszorosítás
#mivel az adatokból sosem elég, a lenti pár sor a már meglévőekből készít újakat

data_transforms = {
    #tanító adatokon végzett műveletek: forgatás, véletlenszerű kivágás stb.
    'train': transforms.Compose([
          transforms.RandomResizedCrop(256),
          transforms.CenterCrop(256),
          transforms.RandomVerticalFlip(),
          transforms.RandomHorizontalFlip(),
          transforms.Resize([512, 384]),

          #képeket tenzoroknak tekintjük, felfoghatók több dimenziós mátrixokként, ahol 3 RGB komponensen van nxm-es mérete
          transforms.ToTensor(),
          transforms.Normalize([0.45, 0.45, 0.45], [0.25, 0.25, 0.25])

    ]),
    'val': transforms.Compose([
          transforms.ToTensor(),
          transforms.Normalize([0.45, 0.45, 0.45], [0.25, 0.25, 0.25])
    ]),
}

In [ ]:
data_dir = '/content/drive/MyDrive/Datasets/IKM_Dataset'

image_datasets = {
    x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
    for x in ['train', 'val']
}   #átlépegetünk a képeken, miközben elvégezzük a transzformálásokat, mindegyikre a megfelelőt

dataloaders = {
    x: torch.utils.data.DataLoader(image_datasets[x], batch_size = batch_size, shuffle = True, num_workers = 2)
    for x in ['train', 'val']
}   #DataLoaderbe tölti az áttranszformált képeket egy olyan objektumba, melyből a háló az adatokat kéri le

dataset_sizes = {
    x: len(image_datasets[x]) for x in ['train', 'val']
}    # -> tanító és validáló képek számait tárolja

class_names = image_datasets['train'].classes #az osztályok neveit tárolja (az általunk megadott mappák nevei -> babel-ek/címkék), így ellenőrzi vissza, hogy helyesen tippelt-e

In [ ]:
#pár kép megjelenítése: ellenőrzés, hogy valóban csatlakoztunk-e a mappánkhoz
def imshow(inp, title = None):
  """Imshow for Tensor."""
  inp = inp.numpy().transpose((1,2,0))    
  mean = np.array([0.45, 0.45, 0.45])
  std = np.array([0.25, 0.25, 0.25])

  inp = std * inp + mean
  inp = np.clip(inp, 0, 1)

  plt.imshow(inp)
  if title is not None:
      plt.title(title)
  plt.pause(0.001)      #kód futását késlelteti ennyi másodperccel -> megvárja az ábrák frissülését

inputs, classes = next(iter(dataloaders['train']))    # Megtekinthetünk egy csomagot (batch)

out = torchvision.utils.make_grid(inputs)
imshow(out, title=[class_names[x] for x in classes])    # batch mérettel tudjuk változtatni, hogy hány képet mutasson, tehát imshow egy db batch méretét mutatja!

#mindig mások a képek, mert shuffle = True van beállítva a dataloaderben, ez mindig véletlenszerűen választja össze a képeket egy csomaggá, ezzel növelve az általánosító képességet

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):

    #mérjük a tanítás/validálás idejét
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # minden epoch-nak lesz egy tanító és egy validáló fázisa
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # tanuló mód
            else:
                model.eval()   # kiértékelési mód

            #kíváncsiak vagyunk a pontosságra és hibára
            running_loss = 0.0
            running_corrects = 0

            #végiglépegetünk a megfelelő ciklus adatain
            for inputs, labels in dataloaders[phase]:

                # cuda-ra tesszük a háló be-és kimeneteit, ahogy magát a hálót is
                inputs = inputs.to(device)
                labels = labels.to(device)

                # nullázzuk a gradienseket
                optimizer.zero_grad()

                # előterjesztés
                # ha tanítunk, akkor számoljuk a gradienseket 
                with torch.set_grad_enabled(phase == 'train'):

                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # hátraterjesztés
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statisztikák
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            #százalékot számolunk
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            accs.append(epoch_acc)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # mindig a legkisebb lokális minimumba került modellt mentjük el, valamint a teljes tanítás után a legjobbat tartjuk meg
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # betöltjük a legjobb modell súlyait
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
#pár kápre megnézzük, hogy hogyan szerepel

num_images = 6
#ennyi képre ellenőrzünk, matplot miatt viszont minél több ez a szám, annál kisebbek lesznek a megjelenített képek

def visualize_model(model, num_images=num_images):
    was_training = model.training
    model.eval()    # kiértékelési módban ellenőrzünk, nem tanulásiban

    images_so_far = 0
    fig = plt.figure()

    # nem kell gradienst számolnunk
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):   # FONTOS: validációs/tesztelő halmazból ellenőrzünk

            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):   # batch_size-szor fut le
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return

        model.train(mode=was_training)

RESNET (A hátraterjesztés során végrehajtandó számtalan szorzásból eredő numerikus problémák orvosolhatók vele)

In [ ]:
model_ft = models.resnet18(pretrained=True)
model_name = 'resnet18'

In [ ]:
model_ft = models.resnet34(pretrained=True)
model_name = 'resnet34'

In [ ]:
model_ft = models.resnet50(pretrained=True)
model_name = 'resnet50'

DENSENET
(ResNet modellel ekvivalens eredményeket lehet elérni kb. feleannyi számítás és paraméter árán)

In [ ]:
model_ft = models.densenet121(pretrained=True)
model_name = 'densenet121'

In [ ]:
model_ft = models.densenet201(pretrained=True)
model_name = 'densenet201'

VGG (Nagyon sokat foglal, sok paramétere van és lassan tanítható, így csak az esetleges pontosság növekedés elérésének érdekében éri meg!)

In [ ]:
model_ft = models.vgg16(pretrained=True)
model_name = 'vgg16'

In [ ]:
num_ftrs = model_ft.fc.in_features

# 6 kimenet: papír, kartondoboz (a megfelelő általánosítóképesség végett), műanyag, fém, üveg, vegyes
model_ft.fc = nn.Linear(num_ftrs, class_num)    
model_ft = model_ft.to(device)

In [ ]:
model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

learning_rate = 0.001
optimizer_ft = optim.SGD(model_ft.parameters(), lr=learning_rate, momentum=0.9)

num_epochs = 25
scheduler = lr_scheduler.CosineAnnealingLR(optimizer_ft, learning_rate, 1e-7)

TANÍTÁS

In [ ]:
#kijelezzük, hogy épp milyen típusú hálót tanítunk
print(f'Using model type: {model_name}\n\n')
model_ft = train_model(model_ft, criterion, optimizer_ft, scheduler, num_epochs=num_epochs)

In [ ]:
#nézzük meg, hogy teljesít
visualize_model(model_ft)

MENTÉSEK/VISSZATÖLTÉSEK:

In [ ]:
model_save_name = 'IKM_resnet34_uj.pt'

In [ ]:
model_save_name = 'IKM_densenet201_uj.pt'

In [ ]:
path = f'/content/drive/MyDrive/{model_save_name}'

In [ ]:
torch.save(model_ft.state_dict(), path)

In [ ]:
model_ft = models.resnet34()

model_save_name = 'IKM_resnet34_uj.pt'
path = f'/content/drive/MyDrive/{model_save_name}'

num_ftrs = model_ft.fc.in_features

model_ft.fc = nn.Linear(num_ftrs, class_num)
model_ft = model_ft.to(device)

model_ft.load_state_dict(torch.load(path))
model_ft.eval()